In [1]:
import os
import pathlib as Path
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random as rdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import datetime

In [2]:
def read_img(files):
    images = []
    labels = []
    for i in files:
        image = cv2.imread(i)
        image = cv2.resize(image,(150,150))
        images.append(image)
        labels.append(os.path.split(os.path.split(i)[0])[1])
    images = np.array(images)
    return images,labels

In [3]:
files = tf.io.gfile.glob('Mainpath3/Train/*/*.jpg')

In [5]:
rdm.shuffle(files)

In [6]:
train_data,train_labels = read_img(files)

In [7]:
val_files = tf.io.gfile.glob('Mainpath3/new_val/*/*.jpg')

In [8]:
rdm.shuffle(val_files)

In [9]:
val_data,val_labels = read_img(val_files)

In [10]:
test_files = tf.io.gfile.glob('Mainpath3/Test/*/*.jpg')

In [11]:
test_data,test_labels = read_img(test_files)

In [12]:
categories = {'Normal':0, 'Diabetes':1, 'Glaucoma':2, 'Cataract':3, 'Other':4,'Pathologocal_Myopia':5,'Macular_Deneration':6,'Hypertension':7}

In [13]:
train_mapped_lables = list(map(lambda x: categories.get(x, x), train_labels))

In [14]:
val_mapped_lables = list(map(lambda x: categories.get(x, x), val_labels))
test_mapped_lables = list(map(lambda x: categories.get(x, x), test_labels))

In [15]:
train_arr = np.array(train_mapped_lables)
val_arr = np.array(val_mapped_lables)
test_arr = np.array(test_mapped_lables)

In [16]:
train_data_dir = 'Mainpath3/Train/'
val_data_dir = 'Mainpath3/new_val/'
test_data_dir = 'Mainpath3/Test/'
batch_size = 32

In [17]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescale validation data (no augmentation for validation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')

Found 6611 images belonging to 5 classes.
Found 660 images belonging to 5 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')

Found 660 images belonging to 5 classes.


In [19]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu',input_shape=(150,150,3)))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(5, activation='softmax'))

In [20]:
model2.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history2 = model2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
)

Epoch 1/30
 33/206 [===>..........................] - ETA: 4:17 - loss: 1.4780 - accuracy: 0.3305

In [ ]:
loss2, accuracy2 = model2.evaluate(test_generator, batch_size= batch_size)